<div class="alert alert-block alert-info text-center">
    <H1> Interactive Portfolio Dashboard </H1>
</div>

<div class="alert alert-block alert-info text-center">
    <H1> PHASE I </H1>
</div>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
def expand_buyselldf(df):
    buysell_df = df.copy()
    buysell_df['transact_val'] = round(buysell_df['quantity'] * buysell_df['price'], 2)

    # Getting the previous row for the ticker
    prev_row = []
    for x, tick in enumerate(buysell_df['ticker']):
        if x == 0:
            prev_row.append(pd.NA)

        else:
            row_tick = buysell_df['ticker'][:x]
            last_occ = row_tick.where(row_tick == tick).last_valid_index()

            if last_occ is not None:
                prev_row.append(last_occ)
            else:
                prev_row.append(pd.NA)

    buysell_df['last_occurrence'] = prev_row
    buysell_df['last_occurrence'] = buysell_df['last_occurrence'].astype('Int64')

    # Getting the cashflow column
    cash_flow = []
    for x, ref in enumerate(buysell_df['type']):
        if ref == 'Buy':
            cash_flow.append(buysell_df['quantity'].iloc[x] * buysell_df['price'].iloc[x] * (-1))
        else:
            cash_flow.append(buysell_df['quantity'].iloc[x] * buysell_df['price'].iloc[x])

    buysell_df['cashflow'] = cash_flow
    buysell_df['cashflow'] = buysell_df['cashflow'].round(2)

    # Getting the previous units and the cumulative units for each row
    buysell_df['prev_units'] = 0.0
    buysell_df['cml_units'] = 0.0
    for x, ref in enumerate(buysell_df['last_occurrence']):
        if ref is pd.NA:
            buysell_df.iat[x,9] = 0
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,10] = buysell_df['quantity'].iloc[x]

        else:
            buysell_df.iat[x,9] = buysell_df['cml_units'].iloc[ref]
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,10] = round(buysell_df['cml_units'].iloc[ref] + buysell_df['quantity'].iloc[x], 4)
            else:
                buysell_df.iat[x,10] = round(buysell_df['cml_units'].iloc[ref] - buysell_df['quantity'].iloc[x], 4)

    # Getting the previous cost, cumulative cost, transtype cost and unit cost for each row
    buysell_df['prev_cost'] = 0.0 # 11
    buysell_df['cml_cost'] = 0.0 # 12
    buysell_df['cost_transact'] = 0.0 # 13
    buysell_df['cost_unit'] = 0.0 # 14

    for x, ref in enumerate(buysell_df['last_occurrence']):
        if ref is pd.NA:
            buysell_df.iat[x,11] = 0
            buysell_df.iat[x,13] = np.nan
            buysell_df.iat[x,14] = np.nan
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,12] = buysell_df['transact_val'].iloc[x]
                buysell_df.iat[x,13] = np.nan
                buysell_df.iat[x,14] = np.nan
            # there should be no SELL on the first row!

        else: # in case last occurrence is not nan
            buysell_df.iat[x,11] = buysell_df['cml_cost'].iloc[ref]
            if buysell_df['type'].iloc[x] == 'Buy':
                buysell_df.iat[x,12] = round(buysell_df['cml_cost'].iloc[ref] + buysell_df['transact_val'].iloc[x], 4)
                buysell_df.iat[x,13] = np.nan
                buysell_df.iat[x,14] = np.nan
            else: # in case SELL
                buysell_df.iat[x,13] = round((buysell_df['quantity'].iloc[x]) / (buysell_df['cml_units'].iloc[ref]) * (buysell_df['cml_cost'].iloc[ref]), 4)
                buysell_df.iat[x,12] = round(buysell_df['cml_cost'].iloc[ref] - buysell_df['cost_transact'].iloc[x], 4)
                buysell_df.iat[x,14] = round(buysell_df['cml_cost'].iloc[ref] / buysell_df['cml_units'].iloc[ref], 4)

    # Getting the realized Gain/Loss and yield %
    buysell_df['gain_loss'] = 0.0
    buysell_df['yield'] = 0.0
    for x, ref in enumerate(buysell_df['type']):
        if ref == 'Sell':
            buysell_df.iat[x,15] = round(buysell_df['transact_val'].iloc[x] - buysell_df['cost_transact'].iloc[x], 4)
            buysell_df.iat[x,16] = round(buysell_df.iat[x,15] / buysell_df['cost_transact'].iloc[x], 4)
    return buysell_df.fillna(0)

def clean_header(df):
    df.columns = df.columns.str.strip().str.lower().str.replace('.', '', regex=False).str.replace('(', \
                '', regex=False).str.replace(')', '', regex=False).str.replace(' ', '_', regex=False).str.replace('_/_', '/', regex=False)
    
def get_now():
    now = datetime.now().strftime('%Y-%m-%d_%Hh%Mm')
    return now

def datetime_maker(df, datecol):
    df[datecol] = pd.to_datetime(df[datecol])

In [ ]:
broker1_raw = pd.read_excel("../inputs/broker1/broker1.xlsx", engine='openpyxl')
broker1_raw.sort_index(inplace=True)
clean_header(broker1_raw)
datetime_maker(broker1_raw, 'time')
broker1_raw['no_of_shares'] = broker1_raw['no_of_shares'].round(4)
broker1_raw['action'].mask(broker1_raw['action'].str.contains('uy'), 'Buy', inplace=True)
broker1_raw['action'].mask(broker1_raw['action'].str.contains('ell'), 'Sell', inplace=True)
buysell_filter = (broker1_raw['action'].str.contains('Buy') | broker1_raw['action'].str.contains('Sell'))
cols_brok1 = ['time', 'action', 'ticker', 'no_of_shares', 'price/share', 'withholding_tax']
broker1_buysell = broker1_raw[buysell_filter][cols_brok1]
broker1_buysell.reset_index(inplace=True, drop=True)
cols_buysell = ['date', 'type', 'ticker', 'quantity', 'price', 'fees']
broker1_buysell.columns = cols_buysell
broker1_buysell['date'] = broker1_buysell['date'].dt.normalize()

In [ ]:
broker2_raw = pd.read_csv("../inputs/broker2/broker2.csv", sep=';')
broker2_raw = broker2_raw[['Date', 'Type', 'Ticker', 'Quantity', 'Price per share', 'FX Rate']]
broker2_raw.columns = cols_buysell
broker2_raw.date = pd.to_datetime(broker2_raw.date, format="%d/%m/%Y %H:%M")
broker2_raw.type = broker2_raw.type.str.capitalize()
broker2_buysell = broker2_raw[broker2_raw.type.str.lower().str.contains('buy|sell')]
broker2_buysell.sort_values(by='date')
broker2_buysell.reset_index(inplace=True, drop=True)

In [ ]:
merged_transactions = broker2_buysell.append(broker1_buysell)
merged_transactions.reset_index(inplace=True, drop=True)
merged_transactions.fillna(0, inplace=True)
merged_transactions.date = merged_transactions.date.dt.date
merged_transactions = merged_transactions.sort_values(by='date').reset_index(drop=True)

In [ ]:
merged_transactions

In [ ]:
final = expand_buyselldf(merged_transactions).sort_values(by='date')

In [ ]:
final.tail(10)

In [ ]:
final['cml_cashflow'] = final['cashflow'].cumsum()*-1

In [ ]:
final['avg_price'] = final['cml_cost']/final['cml_units']

In [ ]:
final.tail(10)

In [ ]:
final.to_excel('../outputs/transactions_all/transactions_finaldf_{}.xlsx'.format(get_now()), index=False)

In [ ]:
# final.groupby(['ticker']).last().to_excel('../outputs/transactions_all/finaldf_lastpositions_{}.xlsx'.format(get_now()))